In [96]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse
import os

# English A&E Waiting Times

NHS England publishes the data <a href="https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2022-23/">here</a> but doesn't offer timeseries on the NHS Trust level. Therefore, we have to download separate files for each month.

In [97]:
## e.g https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2022-23/
index_base = "https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-{year_1}-{year_2}/"

first getting the links for the index pages:

In [98]:
# let's harvest from the most recent page
url = "https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2022-23/"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
# getting all <a> with "A&E Attendances and Emergency Admissions" in the text
ae_index_a = soup.find_all('a')
ae_index_a = [link for link in ae_index_a if "A&E Attendances and Emergency Admissions" in link.text]
# filter for only the links that have a year in them
ae_index_a = [link for link in ae_index_a if "20" in link.text]

then getting the urls of every linked xls

In [99]:
xls_links = []
soups = []
for a in ae_index_a:
    print(f"Fetching from {a.text}")
    print(f"    {a['href']}")
    url = a['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    soups.append(soup)
    links = soup.find_all("a")
    ae_links = [l for l in links if ("Monthly A&E" in l.text or "A&E Week" in l.text) and "xls" in l.text.lower()]
    xls_links.append(ae_links)
    print(f"    Found {len(ae_links)} xls A&E links", end="\n\n")
    time.sleep(1)


Fetching from A&E Attendances and Emergency Admissions 2023-24
    https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2023-24/


KeyboardInterrupt: 

let's load our downloaded xls - only 2015 onward for now

In [115]:
dfs = []
for i, path in enumerate(os.listdir("England")):

    # we only want paths starting with a month name (the 2015 onward date)
    if not (path[0].isalpha() and path[:2] != "AE"):
        continue    

    if i%100 == 0:
        print(f"", end="")
    try:
        df = pd.read_excel(os.path.join("England", path), skiprows=15)
        
        # only want dfs with 'Code' and 'System' columns (the current format)
        if "Code" not in df.columns or "System" not in df.columns:
            continue

        df["source"] = path
        dfs.append(df)
        print(".", end="")
    except:
        print("")
        print(f"x: {path}")
    


....
x: October-2022-AE-by-provider-vM3Lk-revised-110523.xls
......
x: August-2022-AE-by-provider-revised-110523-1Gqhs.xls
.
x: December-2022-AE-by-provider-UScya-revised-110523.xls
..

In [116]:
df = pd.concat(dfs)

In [123]:
df

,Unnamed: 0,Code,System,Type 1 Departments - Major A&E,Type 2 Departments - Single Specialty,Type 3 Departments - Other A&E/Minor Injury Unit,Total attendances,Type 1 Departments - Major A&E.1,Type 2 Departments - Single Specialty.1,Type 3 Departments - Other A&E/Minor Injury Unit.1,...,Percentage in 4 hours or less (type 3),Emergency Admissions via Type 1 A&E,Emergency Admissions via Type 2 A&E,Emergency Admissions via Type 3 and 4 A&E,Total Emergency Admissions via A&E,Other Emergency admissions (i.e not via A&E),Total Emergency Admissions,Number of patients spending >4 hours from decision to admit to admission,Number of patients spending >12 hours from decision to admit to admission,source
0,NaN,-,-,1243898.0,42076.0,682364.0,1968338.0,632299.0,34327.0,607412.0,...,0.961335,364353.0,1464.0,5386.0,371203.0,137576.0,508779.0,142210.0,42767.0,January-2023-AE-by-provider-9wvXO-revised-1105...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,January-2023-AE-by-provider-9wvXO-revised-1105...
2,NaN,QOX,"NHS Bath And North East Somerset, Swindon And ...",15293.0,233.0,9153.0,24679.0,8829.0,233.0,8888.0,...,0.971048,5413.0,0.0,11.0,5424.0,3681.0,9105.0,1728.0,262.0,January-2023-AE-by-provider-9wvXO-revised-1105...
3,NaN,QHG,"NHS Bedfordshire, Luton And Milton Keynes Inte...",22278.0,0.0,14914.0,37192.0,5424.0,0.0,7553.0,...,0.974078,5992.0,0.0,0.0,5992.0,2554.0,8546.0,871.0,36.0,January-2023-AE-by-provider-9wvXO-revised-1105...
4,NaN,QHL,NHS Birmingham And Solihull Integrated Care Board,34258.0,0.0,17614.0,51872.0,20208.0,0.0,17614.0,...,1.000000,10023.0,0.0,0.0,10023.0,5336.0,15359.0,4755.0,1493.0,January-2023-AE-by-provider-9wvXO-revised-1105...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,NaN,QJG,NHS Suffolk And North East Essex Integrated Ca...,21799.0,0.0,12095.0,33894.0,13401.0,0.0,12024.0,...,0.994130,6677.0,0.0,264.0,6941.0,1453.0,8394.0,1601.0,641.0,July-2023-AE-by-provider-DF16G.xls
40,NaN,QXU,NHS Surrey Heartlands Integrated Care Board,26369.0,0.0,7088.0,33457.0,18011.0,0.0,6899.0,...,0.973335,8173.0,0.0,0.0,8173.0,2281.0,10454.0,1639.0,145.0,July-2023-AE-by-provider-DF16G.xls
41,NaN,QNX,NHS Sussex Integrated Care Board,35864.0,1482.0,26720.0,64066.0,22879.0,1468.0,26360.0,...,0.986527,11710.0,30.0,45.0,11785.0,2345.0,14130.0,1676.0,516.0,July-2023-AE-by-provider-DF16G.xls
42,NaN,QUA,NHS Black Country Integrated Care Board,46106.0,1365.0,30267.0,77738.0,28050.0,1159.0,29677.0,...,0.980507,15502.0,0.0,0.0,15502.0,3740.0,19242.0,3882.0,210.0,July-2023-AE-by-provider-DF16G.xls


In [122]:
len(df.source.unique())

13

In [117]:
df.columns

Index(['Unnamed: 0', 'Code', 'System', 'Type 1 Departments - Major A&E',
       'Type 2 Departments - Single Specialty',
       'Type 3 Departments - Other A&E/Minor Injury Unit', 'Total attendances',
       'Type 1 Departments - Major A&E.1',
       'Type 2 Departments - Single Specialty.1',
       'Type 3 Departments - Other A&E/Minor Injury Unit.1',
       'Total Attendances < 4 hours', 'Type 1 Departments - Major A&E.2',
       'Type 2 Departments - Single Specialty.2',
       'Type 3 Departments - Other A&E/Minor Injury Unit.2',
       'Total Attendances > 4 hours', 'Percentage in 4 hours or less (all)',
       'Percentage in 4 hours or less (type 1)',
       'Percentage in 4 hours or less (type 2)',
       'Percentage in 4 hours or less (type 3)',
       'Emergency Admissions via Type 1 A&E',
       'Emergency Admissions via Type 2 A&E',
       'Emergency Admissions via Type 3 and 4 A&E',
       'Total Emergency Admissions via A&E',
       'Other Emergency admissions (i.e not via

In [114]:
dfs[-50].columns

Index(['Unnamed: 0', 'Code', 'System', 'Type 1 Departments - Major A&E',
       'Type 2 Departments - Single Specialty',
       'Type 3 Departments - Other A&E/Minor Injury Unit', 'Total attendances',
       'Type 1 Departments - Major A&E.1',
       'Type 2 Departments - Single Specialty.1',
       'Type 3 Departments - Other A&E/Minor Injury Unit.1',
       'Total Attendances < 4 hours', 'Type 1 Departments - Major A&E.2',
       'Type 2 Departments - Single Specialty.2',
       'Type 3 Departments - Other A&E/Minor Injury Unit.2',
       'Total Attendances > 4 hours', 'Percentage in 4 hours or less (all)',
       'Percentage in 4 hours or less (type 1)',
       'Percentage in 4 hours or less (type 2)',
       'Percentage in 4 hours or less (type 3)',
       'Emergency Admissions via Type 1 A&E',
       'Emergency Admissions via Type 2 A&E',
       'Emergency Admissions via Type 3 and 4 A&E',
       'Total Emergency Admissions via A&E',
       'Other Emergency admissions (i.e not via

In [107]:
df.columns

Index(['Unnamed: 0', 'Code', 'System', 'Type 1 Departments - Major A&E',
       'Type 2 Departments - Single Specialty',
       'Type 3 Departments - Other A&E/Minor Injury Unit', 'Total attendances',
       'Type 1 Departments - Major A&E.1',
       'Type 2 Departments - Single Specialty.1',
       'Type 3 Departments - Other A&E/Minor Injury Unit.1',
       'Total Attendances < 4 hours', 'Type 1 Departments - Major A&E.2',
       'Type 2 Departments - Single Specialty.2',
       'Type 3 Departments - Other A&E/Minor Injury Unit.2',
       'Total Attendances > 4 hours', 'Percentage in 4 hours or less (all)',
       'Percentage in 4 hours or less (type 1)',
       'Percentage in 4 hours or less (type 2)',
       'Percentage in 4 hours or less (type 3)',
       'Emergency Admissions via Type 1 A&E',
       'Emergency Admissions via Type 2 A&E',
       'Emergency Admissions via Type 3 and 4 A&E',
       'Total Emergency Admissions via A&E',
       'Other Emergency admissions (i.e not via

In [ ]:
df = pd.read_excel("England/July-2023-AE-by-provider-DF16G.xls", skiprows=15)

In [ ]:
df

,Unnamed: 0,Code,System,Type 1 Departments - Major A&E,Type 2 Departments - Single Specialty,Type 3 Departments - Other A&E/Minor Injury Unit,Total attendances,Type 1 Departments - Major A&E.1,Type 2 Departments - Single Specialty.1,Type 3 Departments - Other A&E/Minor Injury Unit.1,...,Percentage in 4 hours or less (type 2),Percentage in 4 hours or less (type 3),Emergency Admissions via Type 1 A&E,Emergency Admissions via Type 2 A&E,Emergency Admissions via Type 3 and 4 A&E,Total Emergency Admissions via A&E,Other Emergency admissions (i.e not via A&E),Total Emergency Admissions,Number of patients spending >4 hours from decision to admit to admission,Number of patients spending >12 hours from decision to admit to admission
0,NaN,-,-,1379854.0,45513.0,769533.0,2194900.0,840480.0,43664.0,739830.0,...,0.959374,0.961401,391312.0,1383.0,7719.0,400414.0,134752.0,535166.0,109515.0,23934.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,QOX,"NHS Bath And North East Somerset, Swindon And ...",17709.0,313.0,10283.0,28305.0,10699.0,313.0,9989.0,...,1,0.971409,5978.0,0.0,2.0,5980.0,3723.0,9703.0,1245.0,87.0
3,NaN,QHG,"NHS Bedfordshire, Luton And Milton Keynes Inte...",24256.0,0.0,15012.0,39268.0,15467.0,0.0,14864.0,...,-,0.990141,6474.0,0.0,0.0,6474.0,2453.0,8927.0,773.0,0.0
4,NaN,QHL,NHS Birmingham And Solihull Integrated Care Board,37513.0,0.0,18179.0,55692.0,20931.0,0.0,18141.0,...,-,0.997910,11960.0,0.0,1.0,11961.0,5106.0,17067.0,4925.0,1097.0
5,NaN,QUY,"NHS Bristol, North Somerset And South Gloucest...",22855.0,2232.0,7624.0,32711.0,16443.0,2115.0,7368.0,...,0.947581,0.966422,7558.0,10.0,0.0,7568.0,3092.0,10660.0,1153.0,46.0
6,NaN,QU9,"NHS Buckinghamshire, Oxfordshire And Berkshire...",34158.0,5478.0,9288.0,48924.0,22275.0,5461.0,8904.0,...,0.996897,0.958656,9978.0,913.0,54.0,10945.0,5959.0,16904.0,1201.0,123.0
7,NaN,QUE,NHS Cambridgeshire And Peterborough Integrated...,23084.0,0.0,11329.0,34413.0,13036.0,0.0,10753.0,...,-,0.949157,5969.0,0.0,130.0,6099.0,1776.0,7875.0,1749.0,629.0
8,NaN,QYG,NHS Cheshire And Merseyside Integrated Care Board,73441.0,1564.0,45657.0,120662.0,43220.0,1452.0,44373.0,...,0.928389,0.971877,22737.0,79.0,0.0,22816.0,7066.0,29882.0,9011.0,2944.0
9,NaN,QT6,NHS Cornwall And The Isles Of Scilly Integrate...,6862.0,0.0,12765.0,19627.0,3257.0,0.0,12246.0,...,-,0.959342,2254.0,0.0,44.0,2298.0,1238.0,3536.0,1004.0,435.0


# Wales

Welsh data is available as a panel <a href="https://statswales.gov.wales/Catalogue/Health-and-Social-Care/NHS-Hospital-Waiting-Times/emergency-department/performanceagainst12hourwaitingtimestarget-by-hospital">here</a> 

The GUI is clunky and makes it difficult to download >1000 rows. Luckily the API is easy to use. We can get around the 1K limit by requesting 1 year at a time (by filtering for every month code in a given year - 2010m01 -> 2010m12)

with the OpenData api link of:

http://open.statswales.gov.wales/en-gb/dataset/hlth0034

In [ ]:
welsh_data = requests.get("http://open.statswales.gov.wales/en-gb/dataset/hlth0034").json()
welsh_df = pd.DataFrame(welsh_data["value"])

In [ ]:
for year in range(2010, 2024):
    base_url = f"http://open.statswales.gov.wales/en-gb/dataset/hlth0034" # the dataset
    filter = f"?$filter=Measure_SortOrder eq 3 and Target_Code eq '4hr'" # % <= 4 hours
    month_filters = [f"Date_Code eq '{year}m{month:02}'" for month in range(1, 13)]
    filter = filter + " and (" + " or ".join(month_filters) + ")"
    filter = urllib.parse.quote(filter,safe="/?=:$") # encode the filter

    req = requests.get(base_url + filter)
    try:
        with open(f"Wales/4hr_ae_{year}.json", "w") as f:
            f.write(req.text)
        # did we get valid json?
        data = req.json()['value']
        print(f"{year}: {len(data)}")
    except:
        print(f"{year}: error")
    time.sleep(1)

2010: 432
2011: 432
2012: 531
2013: 564
2014: 564
2015: 562
2016: 564
2017: 564
2018: 563
2019: 552
2020: 546
2021: 564
2022: 564
2023: 423


# Scotland
Scotland is seemingly the easiest and offers a full 17K row download <a href="https://www.opendata.nhs.scot/en/dataset/monthly-accident-and-emergency-activity-and-waiting-times"> here </a>.

The only quirk is that locations aren't identified by name - we have to merge them in.
    
- Health Board - <a href="https://geoportal.statistics.gov.uk/datasets/de69d3ad64964917baffd91cea997c1b/explore">csv</a>

In [ ]:
url = "https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/37ba17b1-c323-492c-87d5-e986aae9ab59/download/monthly_ae_activity_202308.csv"
req = requests.get(url)
with open("Scotland/ae.csv", "w") as f:
    f.write(req.text)

# Northern Ireland

Nisra offers panel data <a href="https://datavis.nisra.gov.uk/health/ni-emergency-care-waiting-times-data-apr-jun-22.html">here</a> but it ends at Jun 2022 as of Nov 2023. Since then, data has been released in reports

In [124]:
df = pd.read_csv("Northern_Ireland/Emergency Care Waiting Times.csv")

In [ ]:
df

,Date,Month,Year,Trust,Dept,Type,Under 4 Hours,Between 4 - 12 Hours,Over 12 Hours,Total,Percent Under 4 Hours
0,2022-06-01,Jun 2022,2022/23,Belfast,Mater,Type 1,"1,370","1,231",386,"2,987",45.9%
1,2022-06-01,Jun 2022,2022/23,Belfast,RVH,Type 1,"1,851","3,419","1,766","7,036",26.3%
2,2022-06-01,Jun 2022,2022/23,Belfast,RVH-RAES,Type 2,653,123,0,776,84.1%
3,2022-06-01,Jun 2022,2022/23,Belfast,RBHSC,Type 1,"2,676","1,521",20,"4,217",63.5%
4,2022-06-01,Jun 2022,2022/23,Northern,Antrim Area,Type 1,"3,553","3,109","1,053","7,715",46.1%
...,...,...,...,...,...,...,...,...,...,...,...
3164,2008-04-01,Apr 2008,2008/09,Southern,Daisy Hill,Type 1,"2,773",113,0,"2,886",96.1%
3165,2008-04-01,Apr 2008,2008/09,Southern,South Tyrone,Type 3,"1,332",0,0,"1,332",100.0%
3166,2008-04-01,Apr 2008,2008/09,Western,Altnagelvin Area,Type 1,"3,571",459,0,"4,030",88.6%
3167,2008-04-01,Apr 2008,2008/09,Western,South West Acute,Type 1,"1,922",26,0,"1,948",98.7%


In [ ]:
df.RowKey.value_counts()

0000000000000002    1
0000000000004171    1
0000000000004091    1
0000000000004097    1
0000000000004103    1
                   ..
0000000000002172    1
0000000000002177    1
0000000000002182    1
0000000000002192    1
0000000000006137    1
Name: RowKey, Length: 1000, dtype: int64

In [ ]:
df.Date_Code.value_counts()

2023m08    47
2009m12    36
2011m02    36
2012m01    36
2011m12    36
2011m11    36
2011m10    36
2011m09    36
2011m08    36
2011m07    36
2011m06    36
2011m05    36
2011m04    36
2011m03    36
2011m01    36
2010m01    36
2010m12    36
2010m11    36
2010m10    36
2010m09    36
2010m08    36
2010m07    36
2010m06    36
2010m05    36
2010m04    36
2010m03    36
2010m02    36
2012m02    17
Name: Date_Code, dtype: int64

In [ ]:
base_url + filter

'http://open.statswales.gov.wales/en-gb/dataset/hlth0034?$filter%20=%20Measure_SortOrder%20eq%203'

In [ ]:
url = "http://open.statswales.gov.wales/en-gb/dataset/wimd0022?$filter=Area_Code%20eq%20%27W01001434%27%20and%20Year_Code%20eq%202014"
req = requests.get(url)
df = pd.DataFrame(req.json()["value"])

In [ ]:
df.Area_Code.value_counts()

W01001434    11
Name: Area_Code, dtype: int64

In [ ]:
df

,Data,Hospital_Code,Hospital_ItemName_ENG,Hospital_SortOrder,Hospital_Hierarchy,Hospital_ItemNotes_ENG,Hospital_AltCode1,Date_Code,Date_ItemName_ENG,Date_SortOrder,Date_ItemNotes_ENG,Measure_Code,Measure_ItemName_ENG,Measure_SortOrder,Measure_ItemNotes_ENG,Target_Code,Target_ItemName_ENG,Target_SortOrder,RowKey,PartitionKey
0,9967.000000,7A4W11000029,Cardiff and Vale University Health Board,57,,,W11000029,2009m12,December 2009,43,,Attendances,Total attendances,1,,8hr,8 hour target,2,0000000000000000,
1,9590.000000,7A4W11000029,Cardiff and Vale University Health Board,57,,,W11000029,2009m12,December 2009,43,,Non-breaches,Attendances where patients spend less than the...,2,Attendances where patients spend less than the...,8hr,8 hour target,2,0000000000000001,
2,86.946925,7A4W11000029,Cardiff and Vale University Health Board,57,,,W11000029,2009m12,December 2009,43,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,4hr,4 hour target,1,0000000000000002,
3,96.217518,7A4W11000029,Cardiff and Vale University Health Board,57,,,W11000029,2009m12,December 2009,43,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,8hr,8 hour target,2,0000000000000003,
4,9967.000000,7A4W11000029,Cardiff and Vale University Health Board,57,,,W11000029,2009m12,December 2009,43,,Attendances,Total attendances,1,,4hr,4 hour target,1,0000000000000004,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,88.113119,7A27A2Maj,Major emergency departments,16,7A2W11000025,Major emergency departments - defined as a con...,7A2Maj,2010m04,April 2010,47,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,4hr,4 hour target,1,0000000000000995,
996,3573.000000,7A27A2Min,Other emergency departments/minor injury units,20,7A2W11000025,Other emergency departments/minor injury units...,7A2Min,2010m04,April 2010,47,,Non-breaches,Attendances where patients spend less than the...,2,Attendances where patients spend less than the...,8hr,8 hour target,2,0000000000000996,
997,3639.000000,7A27A2Min,Other emergency departments/minor injury units,20,7A2W11000025,Other emergency departments/minor injury units...,7A2Min,2010m04,April 2010,47,,Attendances,Total attendances,1,,8hr,8 hour target,2,0000000000000997,
998,98.186315,7A27A2Min,Other emergency departments/minor injury units,20,7A2W11000025,Other emergency departments/minor injury units...,7A2Min,2010m04,April 2010,47,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,8hr,8 hour target,2,0000000000000998,


In [ ]:
df.query("Measure_ItemName_ENG == 'Percentage of patients who spend less than the target time in an emergency department'")

,Data,Hospital_Code,Hospital_ItemName_ENG,Hospital_SortOrder,Hospital_Hierarchy,Hospital_ItemNotes_ENG,Hospital_AltCode1,Date_Code,Date_ItemName_ENG,Date_SortOrder,Date_ItemNotes_ENG,Measure_Code,Measure_ItemName_ENG,Measure_SortOrder,Measure_ItemNotes_ENG,Target_Code,Target_ItemName_ENG,Target_SortOrder,RowKey,PartitionKey
2,86.946925,7A4W11000029,Cardiff and Vale University Health Board,57,,,W11000029,2009m12,December 2009,43,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,4hr,4 hour target,1,0000000000000002,
3,96.217518,7A4W11000029,Cardiff and Vale University Health Board,57,,,W11000029,2009m12,December 2009,43,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,8hr,8 hour target,2,0000000000000003,
8,91.555212,7A5xW11000027,Cwm Taf University Health Board,43,,From 1st April 2019 health service provision f...,W11000027,2009m12,December 2009,43,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,4hr,4 hour target,1,0000000000000008,
9,98.264618,7A5xW11000027,Cwm Taf University Health Board,43,,From 1st April 2019 health service provision f...,W11000027,2009m12,December 2009,43,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,8hr,8 hour target,2,0000000000000009,
12,97.110286,7A6W11000028,Aneurin Bevan University Health Board,49,,,W11000028,2009m12,December 2009,43,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,8hr,8 hour target,2,0000000000000012,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,87.438071,7A5x7A5Maj,Major emergency departments,44,7A5xW11000027,Major emergency departments - defined as a con...,7A5Maj,2010m04,April 2010,47,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,4hr,4 hour target,1,0000000000000986,
989,96.278763,7A5x7A5Maj,Major emergency departments,44,7A5xW11000027,Major emergency departments - defined as a con...,7A5Maj,2010m04,April 2010,47,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,8hr,8 hour target,2,0000000000000989,
992,97.770476,7A27A2Maj,Major emergency departments,16,7A2W11000025,Major emergency departments - defined as a con...,7A2Maj,2010m04,April 2010,47,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,8hr,8 hour target,2,0000000000000992,
995,88.113119,7A27A2Maj,Major emergency departments,16,7A2W11000025,Major emergency departments - defined as a con...,7A2Maj,2010m04,April 2010,47,,Percentages,Percentage of patients who spend less than the...,3,Percentage of attendances where patients spend...,4hr,4 hour target,1,0000000000000995,


In [ ]:
df.Measure_ItemName_ENG.value_counts()

Attendances where patients spend less than the target time in an emergency department    334
Total attendances                                                                        333
Percentage of patients who spend less than the target time in an emergency department    333
Name: Measure_ItemName_ENG, dtype: int64

In [ ]:
req

<Response [404]>

In [ ]:
welsh_df.Date_ItemName_ENG.value_counts()

December 2009    216
January 2010     216
February 2010    216
March 2010       216
April 2010       136
Name: Date_ItemName_ENG, dtype: int64

In [ ]:
welsh_df.Hospital_ItemName_ENG.value_counts()

Other emergency departments/minor injury units    220
Major emergency departments                       192
Cardiff and Vale University Health Board           30
Nevill Hall Hospital                               30
Prince Charles Hospital                            30
The Royal Glamorgan Hospital                       30
Powys Teaching Health Board                        30
Wrexham Maelor Hospital                            30
Cwm Taf University Health Board                    30
Ysbyty Glan Clwyd                                  30
Ysbyty Gwynedd                                     30
Wales                                              30
Betsi Cadwaladr University Health Board            30
Bronglais General Hospital                         30
Glangwili General Hospital                         30
Aneurin Bevan University Health Board              30
Hywel Dda University Health Board                  24
Withybush General Hospital                         24
University Hospital of Wales

In [ ]:
for links in xls_links:
    for link in links:
        print(link.text)
        print(f"    {link['href']}")
        req = requests.get(link['href'])
        file_name = link['href'].split("/")[-1]
        with open(f"England/{file_name}", 'wb') as f:
            f.write(req.content)

xls_links

In [ ]:
for a in ae_index_a:
    print(a.text)
    print(a['href'])

A&E Attendances and Emergency Admissions 2023-24
https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2023-24/
A&E Attendances and Emergency Admissions 2022-23
https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2022-23/
A&E Attendances and Emergency Admissions 2021-22
https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2021-22/
A&E Attendances and Emergency Admissions 2020-21
https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2020-21/
A&E Attendances and Emergency Admissions 2019-20
https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2019-20/
A&E Attendances and Emergency Admissions 2018-19
https:

In [ ]:
len(ae_index_pages)

8

In [ ]:
soups = []
xls_links = []

for start_year in range(2010, 2024):
    print(f"Getting data for {start_year}-{start_year+1}")
    url = index_base.format(year_1=start_year, year_2=start_year+1)
    print(f"    {url}")
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    soups.append(soup)
    links = soup.find_all("a")
    ae_links = [l for l in links if "Monthly A&E" in l.text and "xls" in l.text.lower()]
    xls_links.append(ae_links)
    print(f"    Found {len(ae_links)} xls A&E links", end="\n\n")
    time.sleep(2)

Getting data for 2010-2011
    https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2010-2011/
    Found 0 xls A&E links



KeyboardInterrupt: 

In [ ]:
real
https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/weekly-ae-sitreps-2010-11/
https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2010-2011/

In [ ]:
url = "https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-2022-23/"
req = requests.get(url)

In [ ]:
req.content
soup = BeautifulSoup(req.content, 'html.parser')
# get all links
links = soup.find_all("a")

In [ ]:
ae_links = [l for l in links if "Monthly A&E" in l.text and "CSV" in l.text]

In [ ]:
ae_links


[<a href="https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/05/March-2023-revised-110523-lll12c.csv">Monthly A&amp;E March 2023 (Revised 11.05.2023) (CSV, 29K)</a>,
 <a href="https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/05/February-2023-revised-110523-kkk11b.csv">Monthly A&amp;E February 2023 (Revised 11.05.2023) (CSV, 30K)</a>,
 <a href="https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/05/January-2023-revised-110523-jjj10a.csv">Monthly A&amp;E January 2023 (Revised 11.05.2023) (CSV, 30K)</a>,
 <a href="https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/05/December-2022-revised-110523-iii999.csv">Monthly A&amp;E December 2022 (Revised 11.05.2023) (CSV, 30K)</a>,
 <a href="https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/05/November-2022-revised-110523-hhh888.csv">Monthly A&amp;E November 2022 (Revised 11.05.2023) (CSV, 30K)</a>,
 <a href="https://www.england.nhs.uk/statistics/wp-co

In [ ]:
# print all links
for link in links:
    print("")
    print(link.text)
    print("     " + link.get("href"))


Skip to main content
     #main-content

read more about our cookies
     https://www.england.nhs.uk/privacy-policy/

Home
     https://www.england.nhs.uk/

News
     https://www.england.nhs.uk/news/

Publications
     https://www.england.nhs.uk/publication/

Statistics
     https://www.england.nhs.uk/statistics/

Blogs
     https://www.england.nhs.uk/blogs/

Events
     https://www.england.nhs.uk/events/

Contact us
     https://www.england.nhs.uk/contact-us/




     https://www.england.nhs.uk/ourwork/

About us
     https://www.england.nhs.uk/about/

Our work
     https://www.england.nhs.uk/ourwork

Commissioning
     https://www.england.nhs.uk/commissioning/

Get involved
     https://www.england.nhs.uk/participation/

Statistics
     https://www.england.nhs.uk/statistics

Statistical work areas
     https://www.england.nhs.uk/statistics/statistical-work-areas/

2-hour Urgent Community Response
     https://www.england.nhs.uk/statistics/statistical-work-areas/2-hour-urgent-communi

In [ ]:
link = links[0]
link.text

'Skip to main content'